In [284]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import numpy as np

In [335]:
# just sample data
def generate_data(num_samples, num_frames):
    data = []
    labels = []
    for _ in range(num_samples):
        joints_movement = np.random.randn(num_frames, 15, 3)  
        # shape: (num_frames, 15, 3)
        data.append(joints_movement)
        labels.append(np.random.randint(2))
    return torch.tensor(data).float(), torch.tensor(labels)

num_train_samples = 800
num_test_samples = 200
num_frames = 100

train_data, train_labels = generate_data(num_train_samples, num_frames)
test_data, test_labels = generate_data(num_test_samples, num_frames)

In [354]:
# training and evaluation loops
def train_test(train_loader, test_loader, model,
               optimizer=torch.optim.Adam(model.parameters(), lr=0.001), 
               criterion=nn.CrossEntropyLoss(), num_epochs=10):
    for epoch in range(num_epochs):
        for i, (frames, labels) in enumerate(train_loader):
            frames = frames.view(-1, frames.size(1), frames.size(2) * 3)  # Reshape input data
            outputs = model(frames)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 10 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

    with torch.no_grad():
        correct = 0
        total = 0
        for frames, labels in test_loader:
            frames = frames.view(-1, frames.size(1), frames.size(2) * 3)
            outputs = model(frames)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Accuracy: {:.2f}%'.format(100 * correct / total))

In [355]:
# simple rnn binary classifier considering 15 individual joints
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [356]:
joint_model = RNNClassifier(5 * 3 * 3, 32, 4, 2)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_dataset = torch.utils.data.TensorDataset(train_data, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_data, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_test(train_loader, test_loader, joint_model)

Epoch [1/10], Step [10/25], Loss: 0.6992
Epoch [1/10], Step [20/25], Loss: 0.6824
Epoch [2/10], Step [10/25], Loss: 0.6933
Epoch [2/10], Step [20/25], Loss: 0.6922
Epoch [3/10], Step [10/25], Loss: 0.6918
Epoch [3/10], Step [20/25], Loss: 0.6966
Epoch [4/10], Step [10/25], Loss: 0.6890
Epoch [4/10], Step [20/25], Loss: 0.7054
Epoch [5/10], Step [10/25], Loss: 0.6911
Epoch [5/10], Step [20/25], Loss: 0.6906
Epoch [6/10], Step [10/25], Loss: 0.6885
Epoch [6/10], Step [20/25], Loss: 0.6908
Epoch [7/10], Step [10/25], Loss: 0.6953
Epoch [7/10], Step [20/25], Loss: 0.6821
Epoch [8/10], Step [10/25], Loss: 0.6925
Epoch [8/10], Step [20/25], Loss: 0.6947
Epoch [9/10], Step [10/25], Loss: 0.6830
Epoch [9/10], Step [20/25], Loss: 0.6950
Epoch [10/10], Step [10/25], Loss: 0.6813
Epoch [10/10], Step [20/25], Loss: 0.6844
Accuracy: 46.50%


In [387]:
# cell for bodily components
class Component(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(Component, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [388]:
# rnn classifier grouping joints into components
class ComponentsClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(ComponentsClassifier, self).__init__()
        self.right_arm = Component(input_size, hidden_size, num_layers, num_classes)
        self.left_arm = Component(input_size, hidden_size, num_layers, num_classes)
        self.spine = Component(input_size, hidden_size, num_layers, num_classes)
        self.right_leg = Component(input_size, hidden_size, num_layers, num_classes)
        self.left_leg = Component(input_size, hidden_size, num_layers, num_classes)

    def forward(self, x):
        out1 = self.right_arm(x[:, :, :9]) 
        out2 = self.left_arm(x[:, :, 9:18])
        out3 = self.spine(x[:, :, 18:27]) 
        out4 = self.right_leg(x[:, :, 27:36])
        out5 = self.left_leg(x[:, :, 36:45])
        
        out = torch.cat((out1, out2, out3, out4, out5), dim=1)
        return out

In [390]:
input_size = 9
hidden_size = 32
num_layers = 5
num_classes = 2
batch_size = 32
components_class_model = ComponentsClassifier(input_size, hidden_size, num_layers, num_classes)

train_test(train_loader, test_loader, components_class_model)

Epoch [1/10], Step [10/25], Loss: 2.2115
Epoch [1/10], Step [20/25], Loss: 2.1912
Epoch [2/10], Step [10/25], Loss: 2.2461
Epoch [2/10], Step [20/25], Loss: 2.1975
Epoch [3/10], Step [10/25], Loss: 2.1814
Epoch [3/10], Step [20/25], Loss: 2.2121
Epoch [4/10], Step [10/25], Loss: 2.1839
Epoch [4/10], Step [20/25], Loss: 2.2304
Epoch [5/10], Step [10/25], Loss: 2.1686
Epoch [5/10], Step [20/25], Loss: 2.1709
Epoch [6/10], Step [10/25], Loss: 2.1990
Epoch [6/10], Step [20/25], Loss: 2.1615
Epoch [7/10], Step [10/25], Loss: 2.1956
Epoch [7/10], Step [20/25], Loss: 2.2020
Epoch [8/10], Step [10/25], Loss: 2.1759
Epoch [8/10], Step [20/25], Loss: 2.2010
Epoch [9/10], Step [10/25], Loss: 2.1993
Epoch [9/10], Step [20/25], Loss: 2.2391
Epoch [10/10], Step [10/25], Loss: 2.1669
Epoch [10/10], Step [20/25], Loss: 2.1978
Accuracy: 44.00%


In [360]:
def preprocess_data(data, labels):
    # split joints into limb components
    right_arm = data[:, :, :3]
    left_arm = data[:, :, 3:6]
    spine = data[:, :, 6:9]
    right_leg = data[:, :, 9:12]
    left_leg = data[:, :, 12:]
    # concatenate limb components along the last dimension
    processed_data = torch.cat((right_arm, left_arm, spine, right_leg, left_leg))
    processed_labels = torch.cat((labels, labels, labels, labels, labels))
    return processed_data, processed_labels

In [361]:
train_data_processed, train_labels_processed = preprocess_data(train_data, train_labels)
test_data_processed, test_labels_processed = preprocess_data(test_data, test_labels)

components_model = RNNClassifier(3 * 3, 32, 4, 2)  # 5 components with 3 joints each

# print(train_data_processed.size(0),train_data_processed.size(1), train_data_processed.size(2), train_data_processed.size(3))
train_dataset_processed = torch.utils.data.TensorDataset(train_data_processed, train_labels_processed)
train_loader_processed = torch.utils.data.DataLoader(train_dataset_processed,
                                                     batch_size=batch_size, shuffle=True)

test_dataset_processed = torch.utils.data.TensorDataset(test_data_processed,
                                                        test_labels_processed)
test_loader_processed = torch.utils.data.DataLoader(test_dataset_processed,
                                                    batch_size=batch_size, shuffle=False)

train_test(train_loader_processed, test_loader_processed, components_model)

Epoch [1/10], Step [10/125], Loss: 0.7296
Epoch [1/10], Step [20/125], Loss: 0.7083
Epoch [1/10], Step [30/125], Loss: 0.7603
Epoch [1/10], Step [40/125], Loss: 0.7075
Epoch [1/10], Step [50/125], Loss: 0.7185
Epoch [1/10], Step [60/125], Loss: 0.6715
Epoch [1/10], Step [70/125], Loss: 0.7067
Epoch [1/10], Step [80/125], Loss: 0.7076
Epoch [1/10], Step [90/125], Loss: 0.7235
Epoch [1/10], Step [100/125], Loss: 0.7045
Epoch [1/10], Step [110/125], Loss: 0.7583
Epoch [1/10], Step [120/125], Loss: 0.7014
Epoch [2/10], Step [10/125], Loss: 0.7156
Epoch [2/10], Step [20/125], Loss: 0.7597
Epoch [2/10], Step [30/125], Loss: 0.6907
Epoch [2/10], Step [40/125], Loss: 0.7124
Epoch [2/10], Step [50/125], Loss: 0.7086
Epoch [2/10], Step [60/125], Loss: 0.7102
Epoch [2/10], Step [70/125], Loss: 0.6878
Epoch [2/10], Step [80/125], Loss: 0.7039
Epoch [2/10], Step [90/125], Loss: 0.7010
Epoch [2/10], Step [100/125], Loss: 0.6856
Epoch [2/10], Step [110/125], Loss: 0.6990
Epoch [2/10], Step [120/125],